In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import pickle
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

from tqdm import tqdm
from scipy.spatial.distance import hamming, cosine, euclidean

%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
features = pd.read_csv('features.csv', compression='zip')
labels = pd.read_csv('labels.csv', compression='zip')

In [3]:
features.drop(['Unnamed: 0'],axis=1, inplace=True)
labels.drop(['Unnamed: 0'],axis=1, inplace=True)

In [4]:
features.shape

(1330438, 7)

In [5]:
features.head()

,humidity,temperature,pressure,wind direction,wind speed,Latitude,Longitude
0,87.0,284.590217,807.0,268.0,0.0,49.24966,-123.119339
1,88.0,284.588174,849.0,281.0,0.0,49.24966,-123.119339
2,89.0,284.586130,890.0,295.0,0.0,49.24966,-123.119339
3,89.0,284.584087,932.0,309.0,0.0,49.24966,-123.119339
4,90.0,284.582043,973.0,323.0,0.0,49.24966,-123.119339


In [6]:
labels.shape

(1330438, 1)

In [7]:
labels.head()

,weather_description
0,broken clouds
1,broken clouds
2,broken clouds
3,broken clouds
4,broken clouds


In [8]:
labels.describe()

,weather_description
count,1330438
unique,53
top,sky is clear
freq,475715


In [9]:
import keras.models
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [10]:
labels = pd.get_dummies(labels["weather_description"],prefix='wdesc_',drop_first=True)


In [11]:
labels.head()

,wdesc__drizzle,wdesc__dust,wdesc__few clouds,wdesc__fog,wdesc__freezing rain,wdesc__haze,wdesc__heavy intensity drizzle,wdesc__heavy intensity rain,wdesc__heavy intensity shower rain,wdesc__heavy shower snow,...,wdesc__thunderstorm,wdesc__thunderstorm with drizzle,wdesc__thunderstorm with heavy drizzle,wdesc__thunderstorm with heavy rain,wdesc__thunderstorm with light drizzle,wdesc__thunderstorm with light rain,wdesc__thunderstorm with rain,wdesc__tornado,wdesc__very heavy rain,wdesc__volcanic ash
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:

model = keras.Sequential()
model.add(Dense(units=12, input_dim=7, activation='relu'))

num_hidden_layers = 2
hidden_layer_size = 4

for i in range(0, num_hidden_layers):
    model.add(Dense(units=hidden_layer_size, activation='relu'))

model.add(Dense(units=52, activation='softmax'))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
avg_accuracy = []
avg_loss = []

data_size = 100000

adam = keras.optimizers.Adam(learning_rate=.0001)
sgd = keras.optimizers.SGD(learning_rate=.00001, nesterov=True)
rmsProp = keras.optimizers.RMSprop()

for i in range(0,1):
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    metrics = model.fit(features.sample(data_size, random_state=22), labels.sample(data_size, random_state=22), validation_split = .2, batch_size=32, shuffle=True, epochs = 30)
    #metrics = model.fit(features, labels, validation_split = .2, batch_size=32, shuffle=True, epochs = 10)

    avg_loss.append(metrics.history['loss'])
    avg_accuracy.append(metrics.history['accuracy'])
    

Train on 80000 samples, validate on 20000 samples
Epoch 1/30
80000/80000 [==============================] - 9s 112us/step - loss: 6.3936 - accuracy: 0.3488 - val_loss: 3.3772 - val_accuracy: 0.3526
Epoch 2/30
80000/80000 [==============================] - 7s 92us/step - loss: 3.3507 - accuracy: 0.3571 - val_loss: 3.3366 - val_accuracy: 0.3526
Epoch 3/30
80000/80000 [==============================] - 7s 92us/step - loss: 3.3056 - accuracy: 0.3571 - val_loss: 3.2884 - val_accuracy: 0.3526
Epoch 4/30
80000/80000 [==============================] - 8s 103us/step - loss: 3.2515 - accuracy: 0.3571 - val_loss: 3.2314 - val_accuracy: 0.3526
Epoch 5/30
80000/80000 [==============================] - 8s 100us/step - loss: 3.1899 - accuracy: 0.3571 - val_loss: 3.1690 - val_accuracy: 0.3526
Epoch 6/30
80000/80000 [==============================] - 8s 103us/step - loss: 3.1237 - accuracy: 0.3571 - val_loss: 3.1039 - val_accuracy: 0.3526
Epoch 7/30
80000/80000 [==============================] - 10s 11

In [ ]:
print("avg_accuracy is: " + str(np.mean(avg_accuracy)))
print("avg_loss is: " + str(np.mean(avg_loss)))